# Denoising of the final mapped spectra using the trained DAE model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import json

In [ ]:
from ddae1d.paths import PROJECT_ROOT
from ddae1d.model import DiamondDAE1D

In [ ]:
with open("config.json", "r") as f:
    config = json.load(f)

model_filename = config["model_filename"]

## Loading previously trained model

In [ ]:
model = DiamondDAE1D.load_model(PROJECT_ROOT / "models" / model_filename)

## Loading final mapped spectra to denoise

In [ ]:
final_map = np.load(PROJECT_ROOT / "data" / "preprocessed" / "final-map" / "preprocessed-final-map.npy")

In [ ]:
n_x, n_y, _ = final_map.shape

In [ ]:
final_map = final_map.reshape((n_x * n_y, -1))

## L2 normalization of the final mapped spectra

In [ ]:
l2_norms = np.linalg.norm(final_map, axis=1)[:, np.newaxis]
l2_norms_safe = np.where(l2_norms == 0, 1, l2_norms)
final_map_norm = final_map / l2_norms_safe

## Performing denoising

In [ ]:
denoised_final_map_norm = model.predict(final_map_norm)

## Restoring original norms

In [ ]:
denoised_final_map = denoised_final_map_norm * l2_norms_safe

## Saving resulting data

In [ ]:
denoised_final_map = denoised_final_map.reshape((n_x, n_y, -1))

In [ ]:
np.save(PROJECT_ROOT / "data" / "results" / "denoised" / "denoised-final-map.npy", denoised_final_map)